Importing the required library packages

In [1]:
import tensorflow as tf
from tensorflow import keras
#import keras as keras
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import math
from keras.models import Sequential, Model, load_model
from keras.layers import Conv2D, Activation, Input, Dense, Add, Lambda
from keras.losses import MeanSquaredError
#from tensorflow.keras.layers import BatchNormalization, SeparableConv2D, MaxPooling2D, Activation, ReLU, LeakyReLU, Flatten, Dropout, Dense, MaxPool2D
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import load_img
from tensorflow.keras.layers import *
import tensorflow.keras.backend as K
from scipy import ndimage
from skimage import measure, metrics

Squeeze and Excitation (SE)

In [2]:
def squeeze_excitation_block(input, filters):
    x = GlobalAveragePooling2D()(input)
    x = Dense(filters // 16, activation='relu')(x)
    x = Dense(filters, activation='sigmoid')(x)
    x = Reshape((1, 1, filters))(x)
    x = Multiply()([input, x])
    return x

Convolutional Block Attention Module (CBAM)

In [3]:
def cbam_block(input, reduction_ratio=16):
    x = input
    channel_axis = 1 if K.image_data_format() == "channels_first" else -1
    filters = x.shape[channel_axis]

    # Channel Attention Module
    avg_pool = GlobalAveragePooling2D()(x)
    max_pool = GlobalMaxPooling2D()(x)
    avg_pool = Dense(filters // reduction_ratio, activation='relu')(avg_pool)
    max_pool = Dense(filters // reduction_ratio, activation='relu')(max_pool)
    avg_pool = Dense(filters, activation='sigmoid')(avg_pool)
    max_pool = Dense(filters, activation='sigmoid')(max_pool)
    channel_attention = Multiply()([x, avg_pool + max_pool])

    # Spatial Attention Module
    max_pool = Lambda(lambda x: K.expand_dims(K.max(x, axis=3)))(channel_attention)
    avg_pool = Lambda(lambda x: K.expand_dims(K.mean(x, axis=3)))(channel_attention)
    spatial_attention = Concatenate(axis=3)([max_pool, avg_pool])
    spatial_attention = Conv2D(1, (3, 3), padding='same', activation='sigmoid')(spatial_attention)
    spatial_attention = Multiply()([channel_attention, spatial_attention])

    return spatial_attention

NBDNet Model

In [4]:
input = Input(shape=(64, 64, 1))

# Noise estimation subnetwork
x = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(input)
x = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(x)
x = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(x)
x = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(x)
x = Conv2D(3, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(x)

# Non-Blind denoising subnetwork
x = concatenate([x, input])
x = squeeze_excitation_block(x, x.shape[-1])
x = cbam_block(x)
conv1 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(x)
conv2 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv1)

# Add additional convolutional layers
conv3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv2)
conv4 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv3)

pool1 = AveragePooling2D(pool_size=(2, 2), padding='same')(conv4)
conv5 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(pool1)
conv6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv5)
conv7 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv6)

pool2 = AveragePooling2D(pool_size=(2, 2), padding='same')(conv7)
conv8 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(pool2)
conv9 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv8)
conv10 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv9)
conv11 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv10)
conv12 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv11)
conv13 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv12)

upsample1 = Conv2DTranspose(128, (3, 3), strides=2, activation='relu', kernel_initializer='he_normal', padding='same')(conv13)
add1 = Add()([upsample1, conv7])
add1 = squeeze_excitation_block(add1, add1.shape[-1])
add1 = cbam_block(add1)
conv14 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(add1)
conv15 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv14)
conv16 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv15)

upsample2 = Conv2DTranspose(64, (3, 3), strides=2, activation='relu', kernel_initializer='he_normal', padding='same')(conv16)
add2 = Add()([upsample2, conv4])
add2 = squeeze_excitation_block(add2, add2.shape[-1])
add2 = cbam_block(add2)
conv17 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(add2)
conv18 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv17)

out = Conv2D(1, (1, 1), kernel_initializer='he_normal', padding='same')(conv18)
out = Add()([out, input])

model = tf.keras.Model(input, out)

model.compile(optimizer=tf.keras.optimizers.Adam(1e-3), loss='mse')
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 64, 64, 1)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 64, 64, 32)   320         ['input_1[0][0]']                
                                                                                                  
 conv2d_1 (Conv2D)              (None, 64, 64, 32)   9248        ['conv2d[0][0]']                 
                                                                                                  
 conv2d_2 (Conv2D)              (None, 64, 64, 32)   9248        ['conv2d_1[0][0]']               
                                                                                              

Load Model Weights

In [6]:
H1 = model.load_weights('path to model weight')

Import the test image

In [ ]:
img1 = load_img("path to test noisy image", color_mode="grayscale")
img1 = np.asarray(img1)
plt.imshow(img1, cmap='gray', vmin=0, vmax=255)
plt.show()
img1 = img1.astype('float32')
img2 = img1/255.0
h,w = img2.shape
img_noise_pred = img2.reshape(1, h, w, 1)

Prediction

In [ ]:
pred = model.predict(img_noise_pred)
_,h,w,_ = pred.shape
pred_img = pred.reshape(h, w)
pred_256 = pred_img*255.0
pred_256 = pred_256.astype('int32')
plt.imshow(pred_256, cmap='gray', vmin=0, vmax=255)
plt.show()